In [18]:
# Import all the liberaries 
import torch 
import torch.nn as nn 
import pandas as pd
from torch.utils.data import DataLoader , Dataset

In [19]:
# Import minist dataset
df = pd.read_csv("Dataset/Mnist/train.csv")
df = df.iloc[:100000]
df.sample(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
6062,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18899,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16762,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3029,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11446,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# train set split
import numpy as np
from sklearn.model_selection import train_test_split

y = df["label"]
x = df.drop(columns= ["label"])
x_train , x_test , y_train , y_test  = train_test_split(x , y , test_size= 0.2 , random_state= 42)
x_train.shape , y_train.shape

x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


In [31]:
# Create model class
import torch.optim as opt

class NeuralNetwork(nn.Module):
    def __init__(self , feature_count):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(feature_count , 256 , bias= True),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p= 0.4),
            nn.Linear(256 , 128 , bias= True),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p = 0.4),
            nn.Linear(128 , 64 ),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p = 0.4),
            nn.Linear(64 , 10)
        )
        
    def forward(self , features):
        out = self.network(features)
        return out
    


In [32]:
# Create Dataset class
class CustomeDataset(Dataset):
    def __init__(self, featues , lables):
        
        self.features = featues.to(dtype = torch.float32)
        self.features = self.features/255
        self.labels = lables.to(dtype = torch.long)
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self , index):
        return self.features[index] , self.labels[index]


In [33]:
# Declare hyperparamater 
batch_size = 16
lr = 0.01
epochs = 20

In [34]:
# Init model and Dataloder 
dataset_train = CustomeDataset(x_train , y_train)
dataloder_train = DataLoader(dataset_train , shuffle= True , batch_size= batch_size)

# Model
model = NeuralNetwork(x_train.shape[1])

loss_function = nn.CrossEntropyLoss()
opti = opt.Adam(model.parameters() , lr = lr, weight_decay= 1e-4)

In [35]:
from torchinfo import summary
summary(model , input_size=(33600, 784))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [33600, 10]               --
├─Sequential: 1-1                        [33600, 10]               --
│    └─Linear: 2-1                       [33600, 256]              200,960
│    └─BatchNorm1d: 2-2                  [33600, 256]              512
│    └─ReLU: 2-3                         [33600, 256]              --
│    └─Dropout: 2-4                      [33600, 256]              --
│    └─Linear: 2-5                       [33600, 128]              32,896
│    └─BatchNorm1d: 2-6                  [33600, 128]              256
│    └─ReLU: 2-7                         [33600, 128]              --
│    └─Dropout: 2-8                      [33600, 128]              --
│    └─Linear: 2-9                       [33600, 64]               8,256
│    └─BatchNorm1d: 2-10                 [33600, 64]               128
│    └─ReLU: 2-11                        [33600, 64]               --


In [36]:
# traing Loop

for epoch in range(epochs):
    total_epoch_loss = 0
    
    for train_features , train_lables in dataloder_train:
        
        # forward pass
        y_pred = model(train_features)
        
        # Loss calculation
        loss = loss_function(y_pred , train_lables)
        
        # clear gradient 
        opti.zero_grad()
         
        # backpropgation
        loss.backward()
        
        # paramter updates
        opti.step()
        
        total_epoch_loss += loss.item()
    
    print(f"Epoch : {epoch+1} , Loss : {total_epoch_loss/len(dataloder_train)}")
    

Epoch : 1 , Loss : 0.6840712177913104
Epoch : 2 , Loss : 0.5600343158025117
Epoch : 3 , Loss : 0.5567376774088257
Epoch : 4 , Loss : 0.5502525425968425
Epoch : 5 , Loss : 0.5476103959693795
Epoch : 6 , Loss : 0.541609094307891
Epoch : 7 , Loss : 0.5524053821003153
Epoch : 8 , Loss : 0.5466694559689079
Epoch : 9 , Loss : 0.5408433848477545
Epoch : 10 , Loss : 0.5430879700982145
Epoch : 11 , Loss : 0.5457507459216174
Epoch : 12 , Loss : 0.5516801993300517
Epoch : 13 , Loss : 0.5440792318947968
Epoch : 14 , Loss : 0.5433670913445807
Epoch : 15 , Loss : 0.5514682091382288
Epoch : 16 , Loss : 0.5496487689905223
Epoch : 17 , Loss : 0.5440142103976437
Epoch : 18 , Loss : 0.5376077573409392
Epoch : 19 , Loss : 0.5524326012212606
Epoch : 20 , Loss : 0.5451419158634685


In [37]:
model.eval() # for diff behavior in train and eval, like drop out or batchnor


NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# Evalution forTestingData
test_dataset = CustomeDataset(x_test , y_test)
test_dataloder = DataLoader(test_dataset , batch_size= 16 , shuffle= False)

total = 0
correct = 0

with torch.no_grad():
    for test_feat , test_label in test_dataloder:
        
        output = model(test_feat)
        
        _ , pos = torch.max(output , dim= 1)
        
        total += test_label.shape[0]
        
        correct += (pos == test_label).sum().item()
    
    print(f"TestingAccuracy : {correct/total}")
        
        

Accuracy : 0.9308333333333333


In [39]:
# Evalution forTrainingData
test_dataset = CustomeDataset(x_test , y_test)
test_dataloder = DataLoader(test_dataset , batch_size= 16 , shuffle= False)

total = 0
correct = 0

with torch.no_grad():
    for test_feat , test_label in dataloder_train:
        
        output = model(test_feat)
        
        _ , pos = torch.max(output , dim= 1)
        
        total += test_label.shape[0]
        
        correct += (pos == test_label).sum().item()
    
    print(f"TrainingAccuracy : {correct/total}")
        
        

TrainingAccuracy : 0.9352976190476191
